# JijZept Solverでベンチマーク問題を解く(ソルバー間比較)

このノートブックでは、JijZept Solverを使用した最適化計算のベンチマーク実行と結果の可視化を行います。

## JijZept Solver 無償WebAPI版の利用方法

JijZept Solver 無償WebAPI版を用いてこのノートブックを実行するための手順は以下のとおりです。

### 必要なパッケージのインストール

以下のパッケージを事前にインストールしておく必要があります：

In [ ]:
!pip install jijzept-solver jijmodeling ommx-pyscipopt-adapter minto matplotlib

### JijZept Solver 無償WebAPI版の利用申請
JijZept Solver 無償WebAPI版 を利用するには、まず[利用申請フォーム](https://docs.google.com/forms/d/e/1FAIpQLScLTRxXGaN7egRkoYcq2ZvFoFXRyYInsmPXlyxk9pF11E9--g/viewform)から申請を行ってください。

### API認証情報の設定

利用申請により入手した、以下の値を環境変数に設定します：

- `JIJZEPT_SOLVER_SERVER_HOST`: API サーバーのホスト名
- `JIJZEPT_SOLVER_ACCESS_TOKEN`: アクセストークン

このノートブックでは、以下のセルの`os.environ`にて設定できます（"APIサーバーのホスト名", "アクセストークン"を、利用申請で取得した値に置き換えてください）

In [ ]:
import os

os.environ["JIJZEPT_SOLVER_SERVER_HOST"] = "APIサーバーのホスト名"
os.environ["JIJZEPT_SOLVER_ACCESS_TOKEN"] = "アクセストークン"

In [ ]:
import random
import time
import jijmodeling as jm
import jijzept_solver
from ommx_pyscipopt_adapter import OMMXPySCIPOptAdapter
import minto

In [ ]:
# QPLIBを使う場合
qplib = jm.dataset.Qplib()
all = sorted(
    [{"name": name, **stat} for name, stat in qplib.instance_statistics.items()],
    key=lambda v: (v["non_zero"], v["name"]),
)

# バイナリ変数のみ、非ゼロ要素数が10万以下のもの
filtered_instances = [
    v["name"]
    for v in all
    if v["continuous"] == 0 and v["integer"] == 0 and v["non_zero"] <= 100_000
]

In [ ]:
# # MIPLIBを使う場合
# miplib = jm.dataset.Miplib()
# all = sorted(
#     [{"name": name, **stat} for name, stat in miplib.instance_statistics.items()],
#     key=lambda v: (v["non_zero"], v["name"]),
# )

# (1)バイナリ変数のみ、非ゼロ要素数が10万以下のもの
# filtered_instances = [
#   v["name"]
#   for v in all
#   if v["continuous"] == 0 and v["integer"] == 0 and v["non_zero"] <= 100_000
# ]

# (2)整数変数のみ、非ゼロ要素数が10万以下のもの
# filtered_instances = [
#     v["name"]
#     for v in all
#     if v["continuous"] == 0
#     and v["binary"] == 0
#     and v["integer"] > 1
#     and v["integer"] <= 100_000
#     and v["non_zero"] <= 100_000
# ]

In [ ]:
# このノートブックでは、条件に当てはまるインスタンスからランダムに5個選択
random.seed(0)
instance_list = random.sample(filtered_instances, min(5, len(filtered_instances)))

## ベンチマーク実行

選択されたインスタンスを用いてベンチマークを実行します。実験管理にはMINTOを利用します。

In [ ]:
# 実行時間制限を指定（秒）
# JijZept Solver 無償WebAPI版では最大10秒までに制限されています
timelimit_list = [1]  # 1秒
# timelimit_list = [1, 5] # 1秒,5秒 （複数の秒数を指定して比較することも可能）

In [ ]:
# 使用するソルバーごとのCallback関数を定義

# JijZept Solver
def solve_with_jijzept_solver(instance, timelimit, experiment):
    print("-- Solving by jijzept_solver...")
    try:
        solution = jijzept_solver.solve(instance, time_limit_sec=timelimit)
        experiment.log_solution("jijzept_solver", solution)
        return solution
    except Exception as e:
        print(f"(Error: {e})")
        return None


# SCIP(OMMXPySCIPOptAdapter)
def solve_with_scip(instance, timelimit, experiment):
    print("-- Solving by scip...")
    try:
        adapter = OMMXPySCIPOptAdapter(instance)
        scip_model = adapter.solver_input
        scip_model.setParam("limits/time", timelimit)
        scip_model.optimize()
        solution = adapter.decode(scip_model)
        experiment.log_solution("scip", solution)
        return solution
    except Exception as e:
        print(f"(Error: {e})")
        return None


# 上記で定義したソルバーCallbackのリストを作成
solver_callbacks = [
    solve_with_jijzept_solver,
    solve_with_scip,
]

In [ ]:
def run_solver_callback(solver_callback, instance, timelimit, experiment):
    return solver_callback(instance, timelimit, experiment)


# 実験管理に用いるMINTOの設定
experiment = minto.Experiment(name="jijzept_solver_benchmark", auto_saving=True)

# ベンチマーク実行
for instance_name in instance_list:
    # JijModelingを用いてQPLIBインスタンスを読み込む
    problem, instance_data = jm.dataset.Qplib().load(instance_name)
    # MIPLIBを用いる場合は以下
    # problem, instance_data = jm.dataset.Miplib().load(instance_name)

    interpreter = jm.Interpreter(instance_data)
    instance = interpreter.eval_problem(problem)

    for timelimit in timelimit_list:
        with experiment.run():
            print(f"instance_name: {instance_name}")
            print(f"- timelimit: {timelimit}")
            experiment.log_parameter("timelimit", timelimit)
            experiment.log_parameter("instance_name", instance_name)

            # 各ソルバーをコールバック形式で実行
            for solver_callback in solver_callbacks:
                run_solver_callback(solver_callback, instance, timelimit, experiment)

            # APIリクエスト負荷軽減のため1秒待機
            time.sleep(1)
            print("--------------------")

結果は以下のテーブルで確認できます。

In [ ]:
# 実験結果テーブルの確認
run_table = experiment.get_run_table()
run_table

## 結果の可視化

ベンチマーク結果の可視化を行います。
まず可視化用に結果テーブルを整形します。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


def get_solver_results(experiment, time_limit=None):
    run_table = experiment.get_run_table()

    # 時間制限が指定されている場合はフィルタリング
    if time_limit is not None:
        mask = run_table["parameter"]["timelimit"] == time_limit
        run_table = run_table[mask]

    # ソルバー名を取得
    solver_columns = []
    for col in run_table.columns:
        if col[0].startswith("solution_"):
            solver_columns.append(col[0])
    solver_columns = list(dict.fromkeys(solver_columns))

    results = []
    for solver_key in solver_columns:
        # 列名からソルバー名を抽出（例：'solution_scip' → 'SCIP'）
        solver_name = solver_key.replace("solution_", "").upper()

        solver_df = pd.DataFrame(
            {
                "instance": run_table["parameter"]["instance_name"],
                "solver": solver_name,
                "objective": run_table[solver_key]["objective"],
                "feasible": run_table[solver_key]["feasible"],
                "timelimit": run_table["parameter"]["timelimit"],
            }
        )
        results.append(solver_df)

    return pd.concat(results, ignore_index=True)


all_results = get_solver_results(experiment)
all_results

各インスタンスに対する結果を棒グラフで表示します。

ここでは、目的関数値(Objective)の値を表示しています。

In [ ]:
def plot_objective(df, timelimit=None, value_column="objective", ylabel=None):
    if timelimit is not None:
        df = df[df["timelimit"] == timelimit]

    if df.empty:
        print("No data to display")
        return

    pivot_df = df.pivot_table(
        index="instance", columns="solver", values=value_column, aggfunc="first"
    )

    if pivot_df.empty:
        print("No instances or solvers to display")
        return

    plt.figure(figsize=(12, 6))

    # 各ソルバーの棒グラフを描画
    bar_width = 0.3
    x = np.arange(len(pivot_df.index))
    colors = plt.cm.tab10(np.arange(len(pivot_df.columns)))  # ソルバーごとに色を固定
    all_values = pivot_df.values.flatten()
    valid_values = all_values[~pd.isna(all_values)]

    if len(valid_values) > 0:
        y_min = min(valid_values)
        y_max = max(valid_values)
        y_range = y_max - y_min if y_max != y_min else abs(y_max) if y_max != 0 else 1
        y_margin_top = y_range * 0.2
        y_margin_bottom = y_range * 0.1
    else:
        y_min, y_max = 0, 1
        y_margin_top = 0.2
        y_margin_bottom = 0.1

    for i, solver in enumerate(pivot_df.columns):
        solver_color = colors[i]
        first_bar = True

        for j, (instance, value) in enumerate(pivot_df[solver].items()):
            if pd.isna(value):
                # NaNの場合は "NaN" をテキストで表示
                plt.text(
                    j + i * bar_width,
                    0,
                    "NaN",
                    ha="center",
                    va="bottom",
                    fontsize=7,
                    rotation=45,
                )
            else:
                plt.bar(
                    j + i * bar_width,
                    value,
                    bar_width,
                    color=solver_color,
                    alpha=0.8,
                    label=solver if first_bar else "",
                )
                first_bar = False
                plt.text(
                    j + i * bar_width,
                    value + abs(value) * 0.03,
                    f"{value:.2f}",
                    ha="center",
                    va="bottom",
                    fontsize=7,
                    rotation=45,
                )

    plt.xlim(-0.5, len(pivot_df.index) - 0.5 + bar_width * len(pivot_df.columns))
    plt.ylim(y_min - y_margin_bottom, y_max + y_margin_top)
    plt.xticks(
        x + bar_width * (len(pivot_df.columns) - 1) / 2, pivot_df.index, rotation=0
    )
    plt.ylabel(ylabel if ylabel else value_column.title())
    plt.legend()
    plt.tight_layout()
    plt.show()


for timelimit in timelimit_list:
    print(f"\n時間制限 {timelimit}秒 の結果:")
    timelimit_results = all_results[all_results["timelimit"] == timelimit]
    plot_objective(
        timelimit_results,
        timelimit=timelimit,
        value_column="objective",
        ylabel="Objective Value",
    )

次に、実行可能解を得られた割合（Feasible Rate）を表示します。
ここでは、以下により算出しています。
```
Feasible Rate = 実行可能解を得られたインスタンス数 / 総インスタンス数 (×100%)
```
注意点：上記の方法に依るため、インスタンスが当該ソルバーに対応していないことによるエラーや、その他実行時の予期せぬエラー等により実行可能解が得られなかった場合も、失敗(infeasible)に含まれます。

In [ ]:
def calculate_feasible_rates(experiment, time_limit=None):
    run_table = experiment.get_run_table()

    if time_limit is not None:
        mask = run_table["parameter"]["timelimit"] == time_limit
        run_table = run_table[mask]

    solver_columns = []
    for col in run_table.columns:
        if col[0].startswith("solution_"):
            solver_columns.append(col[0])
    solver_columns = list(dict.fromkeys(solver_columns))

    total = len(run_table)

    print("=== Feasible Rate ===")
    if time_limit:
        print(f"時間制限: {time_limit}秒")
    print(f"総インスタンス数: {total}")

    for solver_key in solver_columns:
        solver_name = solver_key.replace("solution_", "")
        feasible_count = run_table[solver_key]["feasible"].sum()
        rate = feasible_count / total * 100 if total > 0 else 0
        print(f"{solver_name}: {feasible_count}/{total} ({rate:.1f}%)")


for timelimit in timelimit_list:
    calculate_feasible_rates(experiment, time_limit=timelimit)

Feasible Rateを円グラフで表示します

In [ ]:
def plot_feasible_rate_pie_chart(df, timelimit=None, figsize=(15, 5)):
    if timelimit is not None:
        df = df[df["timelimit"] == timelimit]

    if df.empty:
        print("No data to display")
        return

    solvers = df["solver"].unique()

    if len(solvers) == 0:
        print("No solvers to display")
        return

    fig, axes = plt.subplots(1, len(solvers), figsize=figsize)
    if len(solvers) == 1:
        axes = [axes]

    for i, solver in enumerate(solvers):
        solver_data = df[df["solver"] == solver]
        feasible_count = solver_data["feasible"].sum()
        total_count = len(solver_data)
        infeasible_count = total_count - feasible_count

        axes[i].pie(
            [feasible_count, infeasible_count],
            labels=[f"Feasible ({feasible_count})", f"Infeasible ({infeasible_count})"],
            colors=["#2ca02c", "#d62728"],
            autopct="%1.1f%%",
            startangle=90,
        )
        rate = feasible_count / total_count if total_count > 0 else 0
        axes[i].set_title(f"{solver}\nSuccess Rate: {rate:.1%}")

    fig.suptitle(f"Feasible Rate{f' (Time Limit: {timelimit}s)' if timelimit else ''}")
    plt.tight_layout()
    plt.show()


for timelimit in timelimit_list:
    plot_feasible_rate_pie_chart(all_results, timelimit=timelimit)